# Data

In [62]:
reference_assembly = '../data/NC_059190.1_ref.fna'
reference_annotaton = '../data/NC_059190.1_RefSeq.gff' 
# nanopore_reads 
geneid_parameters = "../data/Daphnia_magna.param"
isoquant_models = "../data/NC_059190.1_isquant_models.gff"

# Get geneid

In [46]:
! git clone https://github.com/guigolab/geneid.git

Cloning into 'geneid'...
remote: Enumerating objects: 21502, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 21502 (delta 0), reused 0 (delta 0), pack-reused 21499 (from 1)
Receiving objects: 100% (21502/21502), 119.37 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (11251/11251), done.


In [47]:
! cd geneid/ && make

mkdir ./objects; 
gcc -c -I./include -Wall -O3 ./src/geneid.c -o ./objects/geneid.o 
gcc -c -I./include -Wall -O3 ./src/BackupGenes.c -o ./objects/BackupGenes.o
gcc -c -I./include -Wall -O3 ./src/PeakEdgeScore.c -o ./objects/PeakEdgeScore.o
gcc -c -I./include -Wall -O3 ./src/GetTranscriptTermini-usingslopes.c -o ./objects/GetTranscriptTermini-usingslopes.o
gcc -c -I./include -Wall -O3 ./src/BuildAcceptors.c -o ./objects/BuildAcceptors.o
gcc -c -I./include -Wall -O3 ./src/BuildU12Acceptors.c -o ./objects/BuildU12Acceptors.o
gcc -c -I./include -Wall -O3 ./src/BuildDonors.c -o ./objects/BuildDonors.o
gcc -c -I./include -Wall -O3 ./src/BuildInitialExons.c -o ./objects/BuildInitialExons.o
gcc -c -I./include -Wall -O3 ./src/BuildInternalExons.c -o ./objects/BuildInternalExons.o
gcc -c -I./include -Wall -O3 ./src/BuildZeroLengthExons.c -o ./objects/BuildZeroLengthExons.o
./src/BuildZeroLengthExons.c: In function ‘BuildZeroLengthExons’:
./src/BuildZeroLengthExons.c:62:14: warning: variable ‘js

In [48]:
! ./geneid/bin/geneid

Error: One filename is required (DNA sequence, Fasta format)
NAME
	geneid - a program to annotate genomic sequences
SYNOPSIS
	geneid	[-bdaefitnxszru]
		[-TDAZU]
		[-p gene_prefix]
		[-G] [-3] [-X] [-M] [-m]
		[-WCF] [-o]
		[-j lower_bound_coord]
		[-k upper_bound_coord]
		[-N numer_nt_mapped]
		[-O <gff_exons_file>]
		[-R <gff_annotation-file>]
		[-S <gff_homology_file>]
		[-P <parameter_file>]
		[-E exonweight]
		[-V evidence_exonweight]
		[-Bv] [-h]
		<locus_seq_in_fasta_format>
RELEASE
	geneid v 1.4



In [64]:
# create a valiable with geneid executable path
# in a real case scenario, add it to your PATH
geneid = './geneid/bin/geneid'

# Predict genes with geneid

In [65]:
! time $geneid -3P $geneid_parameters $reference_assembly > ../results/geneid_simple.gff3


real	0m8.297s
user	0m7.575s
sys	0m0.079s


#### Check simple stats

In [66]:
! grep -v '#' ../results/geneid_simple.gff3 | cut -f 3 | sort | uniq -c 

  13296 CDS
  13296 exon
   2914 gene
   2914 mRNA


In [67]:
! bash ../scripts/extract_features.sh 

Usage: ../scripts/extract_features.sh <gff_file> <fasta_file/genome_size_in_bp>


In [68]:
! bash ../scripts/extract_features.sh ../results/geneid_simple.gff3 $reference_assembly

Feature	Features_Count	Total_Feature_Length	Average_Feature_Length	Genome_Percentage
mRNA	2914	7203206	2471.93	69.98
exon	13296	3463047	260.46	33.65
CDS	13296	3463047	260.46	33.65
gene	2914	7203206	2471.93	69.98


In [69]:
13296/2914

4.562800274536719

# Build Transcitps models with IsoQuant

#### The Isoquant Output

#### Check simple stats

In [6]:
! grep -v '#' $isoquant_models  | cut -f 3 | sort | uniq -c 

5718 exon
 797 gene
1349 transcript


In [10]:
! bash ../scripts/extract_features.sh $isoquant_models $reference_assembly

Feature	Features_Count	Total_Feature_Length	Average_Feature_Length	Genome_Percentage
exon	5718	1185519	207.33	11.52
transcript	1349	5689555	4217.61	55.28
gene	797	3950119	4956.23	38.38


# Inform geneid with IsoQuant models 

### Prepare models

#### Get longest transcript (check --no_check)

In [33]:
! agat_sp_keep_longest_isoform.pl --gff ../data/NC_059190.1_isquant_models.gff -o ../intermediate/isoquant_longest.gff

docker: Error response from daemon: invalid volume specification: '/host_mnt/Users/fzanarello/work/projects/summer_school/practicals/day2/fabio/notebook:~': invalid mount config for type "bind": invalid mount path: '~' mount path must be absolute.
See 'docker run --help'.


In [11]:
! bash ../scripts/extract_features.sh ../intermediate/isoquant_longest.gff ../data/NC_059190.1_ref.fna

Feature	Features_Count	Total_Feature_Length	Average_Feature_Length	Genome_Percentage
exon	3546	783245	220.88	7.61
gene	797	3950119	4956.23	38.38
transcript	797	3294312	4133.39	32.01


#### Get exons

In [13]:
! awk '$3 == "exon"' ../intermediate/isoquant_longest.gff > ../intermediate/isoquant_exons.gff

In [14]:
! gawk 'BEGIN { FS=OFS="\t" } /^#/ { print; next } { match($9, /transcript_id=([^;]+)/, a); $9 = a[1]; print }' ../intermediate/isoquant_exons.gff > ../intermediate/isoquant_exons_geneid.gff 

In [15]:
! bash ../scripts/extract_features.sh ../intermediate/isoquant_exons_geneid.gff  ../data/NC_059190.1_ref.fna

Feature	Features_Count	Total_Feature_Length	Average_Feature_Length	Genome_Percentage
exon	3546	783245	220.88	7.61


#### Label with geneid categories

In [64]:
from collections import defaultdict
import csv

# Input/output files
input_file = "../intermediate/isoquant_exons_geneid.gff"
output_file = "../intermediate/isoquant_exons_geneid_class.gff"

# Group exons by transcript
transcript_exons = defaultdict(list)

with open(input_file) as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split('\t')
        transcript_id = parts[8].strip()  # Assuming 9th column is just the transcript name
        transcript_exons[transcript_id].append(parts)

# Process and label exons
with open(output_file, 'w', newline='') as out:
    writer = csv.writer(out, delimiter='\t', quoting=csv.QUOTE_NONE, escapechar='\\')

    for transcript, exons in transcript_exons.items():

        strand = exons[0][6]
        
        for i, exon in enumerate(exons):
            if i == 0 :
                if strand == '+':
                    exon[2] = "First"
                else:
                    exon[2] = "Terminal"                 
            elif i == len(exons) - 1:
                if strand == '+':
                    exon[2] = "Terminal"
                else:
                    exon[2] = "First"
            else:
                exon[2] = "Internal"
            writer.writerow(exon)

In [71]:
! grep '+' ../intermediate/isoquant_exons_geneid_class.gff > ../intermediate/isoquant_exons_geneid_class_FW.gff

In [72]:
! grep '' ../intermediate/isoquant_exons_geneid_class.gff > ../intermediate/isoquant_exons_geneid_class_RW.gff

### Predict with geneid and -R parameter

In [73]:
! time geneid -3P $geneid_parameters -R ../intermediate/isoquant_exons_geneid_class_FW.gff $reference_assembly > ../results/geneid_fw.gff3

Error: Order violation: annotations (starting position 149468):
-->NC_059190.1	IsoQuant	First	148639	148954	.	+	.	transcript8984.NC_059190.1.nnic


geneid -3P ../data/Daphnia_magna.param -R  ../data/NC_059190.1_ref.fna >   0.06s user 0.08s system 91% cpu 0.155 total


#### Check simple stats

# Run full comparison with compann-nf

### Get reference annotation

#### Get info about the taxon

#### Get annotation for the species of iterest

#### Prepare input

### Run Pipeline

### Inspect results